In [ ]:
!pip -q install transformers accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

In [ ]:
prompt = """
Explain photosynthesis for a non-scientist.

Return ONLY this template and fill it:

- Bullet 1:
- Bullet 2:
- Bullet 3:
- Bullet 4:
- Bullet 5:

Rules:
- Replace each bullet with one short sentence (<= 12 words).
- Keep exactly 5 lines starting with "- ".
- No extra text before or after.
""".strip()

In [ ]:
def generate(prompt, do_sample=False, temperature=0.7, top_p=0.9, max_new_tokens=120, seed=0):
    set_seed(seed)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
print("=== GREEDY ===\n")
print(generate(prompt, do_sample=False))

In [ ]:
print("\n=== SAMPLING RUNS (temp=0.7, top_p=0.9) ===\n")
for i in range(3):
    print(f"\nRun {i}")
    print(generate(prompt, do_sample=True, temperature=0.7, top_p=0.9, seed=i))